# Convert Hands CNN and Detector from ONNX to TFLite

### Hands CNN + Detector Conversion

In [2]:
import sys
import tensorflow as tf
import onnx_tf
import onnx
import numpy as np

sys.path.append("D:\Programming\cybertruck")

In [ ]:
model_name = 'hands_detector'

### Now Prepare the Onnx model for tensorflow and then convert to tflite

https://medium.com/@zergtant/convert-pytorch-model-to-tf-lite-with-onnx-tf-232a3894657c

In [ ]:
def convert_onnx_to_tflite(model_name):
    # Load  ONNX model
    onnx_model = onnx.load(f'{model_name}.onnx')

    # Convert ONNX model to TensorFlow format
    tf_model = onnx_tf.backend.prepare(onnx_model)
    # Export  TensorFlow  model 
    tf_model.export_graph(f'{model_name}.tf')

    # Then convert from TF to TFLite
    converter = tf.lite.TFLiteConverter.from_saved_model(f'{model_name}.tf')
    tflite_model = converter.convert()
    open(f'{model_name}.tflite', 'wb').write(tflite_model)

### Now test the converted model

https://www.tensorflow.org/lite/guide/inference
https://www.tensorflow.org/lite/guide/inference#run_inference_with_dynamic_shape_model

Note that by default, this model takes a batch size of 1. Which works out well since we want real time performance

In [5]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="hands_cnn.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[  1   3 224 224]
[[ 0.38875434 -0.27507943 -0.48259985 -0.04473732  0.13935351 -0.16089939
   0.00969524  0.24680549  0.30112687  0.10554482]]


In [6]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path="raw_cnn.tflite")

# Resize input shape for dynamic shape model and allocate tensor
interpreter.resize_tensor_input(interpreter.get_input_details()[0]['index'], [5, 3, 224 , 224])
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details) # Here we can see the input size has changed to (5, 3, 299, 299) 
print(output_details) # The output has the corresponding changes

input_data = np.array(np.random.random_sample((5, 3, 224, 224)), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  5,   3, 224, 224]), 'shape_signature': array([ -1,   3, 299, 299]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'PartitionedCall:0', 'index': 360, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[ 0.62110627  1.277077   -0.34690422 -0.1362839  -2.0667741  -0.65724695
  -0.00231836  0.6305507   1.2859055  -0.79382193]
 [ 0.62159884  1.2866197  -0.27352217 -0.14923137 -2.018711   -0.6308625
  -0.03146842  0.60975784  1.1814307  -0.7074548 ]
 [ 0.54732144  1.1805639  -0.34844688 -0.11515829 -1.9153498  -0.5589704
